In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import copy
import datetime as dt

In [2]:
def CAGR(df):
    """Function to calculate the cumulative annual growth rate of a trading strategy

    Args:
        df (pandas dataframe): stock data

    Returns:
        CAGR: cumulative annual growth return of strategy
    """
    df_copy = df.copy()
    df_copy["cum_return"] = (1+df["mon_ret"]).cumprod()
    n = len(df_copy)/12
    CAGR = (df_copy["cum_return"].tolist()[-1])**(1/n) - 1
    return CAGR

def volatility(df):
    """Function to calculate the annualized volatility of a trading strategy

    Args:
        df (pandas dataframe): stock data

    Returns:
        vol: volatility of strategy
    """
    df_copy = df.copy()
    vol = df_copy["mon_ret"].std() * np.sqrt(12)
    return vol

def sharpe(df, rf):
    """Function to calculate sharpe ratio

    Args:
        df (pandas dataframe): stock data
        rf (float): risk free rate (returns from benchmark)
    """
    df_copy = df.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr

In [3]:
# Download historical data (monthly) for DJI constituent stocks
tickers = ["MMM","AXP","T","BA","CAT","CVX","CSCO","KO", "XOM","GE","GS","HD",
           "IBM","INTC","JNJ","JPM","MCD","MRK","MSFT","NKE","PFE","PG","TRV",
           "UTX","UNH","VZ","V","WMT","DIS"]

ohlc_mon = {} # directory with ohlc value for each stock            
start = dt.datetime.today()-dt.timedelta(1900)
end = dt.datetime.today()

# looping over tickers and creating a dataframe with close prices
for ticker in tickers:
    ohlc_mon[ticker] = yf.download(ticker,start,end,interval='1mo')
    ohlc_mon[ticker].dropna(inplace=True,how="all")
 
tickers = ohlc_mon.keys() # redefine tickers variable after removing any tickers with corrupted data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
# calculating monthly return for each stock and consolidating return info by 
# stock in a separate dataframe
ohlc_dict = copy.deepcopy(ohlc_mon)
return_df = pd.DataFrame()
for ticker in tickers:
    # percentage change in adjusted close each month
    ohlc_dict[ticker]["mon_ret"] = ohlc_dict[ticker]["Adj Close"].pct_change()

    # add to return dictionary
    return_df[ticker] = ohlc_dict[ticker]["mon_ret"]

return_df.head()

,MMM,AXP,T,BA,CAT,CVX,CSCO,KO,XOM,GE,...,NKE,PFE,PG,TRV,UTX,UNH,VZ,V,WMT,DIS
Date,,,,,,,,,,,,,,,,,,,,,
2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-11-01,0.056258,0.026876,0.099178,0.072951,0.045596,0.026749,0.101678,-0.004567,-0.000720,-0.092758,...,0.098745,0.034227,0.050085,0.023556,NaN,0.085387,0.075841,0.023732,0.113618,0.071670
2017-12-01,-0.027073,0.016375,0.068719,0.071154,0.116401,0.061954,0.026810,0.010622,0.013496,-0.045927,...,0.035253,0.008024,0.021002,0.000516,NaN,-0.030528,0.040087,0.014495,0.015633,0.025663
2018-01-01,0.064282,0.000906,-0.036780,0.201621,0.032999,0.001278,0.084595,0.037271,0.043759,-0.066954,...,0.094270,0.022639,-0.060296,0.111240,NaN,0.074027,0.021538,0.089546,0.085183,0.018921
2018-02-01,-0.059840,-0.015553,-0.013637,0.022124,-0.045658,-0.107140,0.086037,-0.091826,-0.132417,-0.127396,...,-0.017443,-0.019708,-0.083639,-0.072839,NaN,-0.044852,-0.107014,-0.010384,-0.155629,-0.050704


In [ ]:
# function to calculate portfolio return iteratively
def pflio(return_df, number_held_stocks, number_stocks_to_cut):
    """Returns cumulative portfolio return

    Args:
        return_df (pandas dataframe): dataframe with monthly return info for 
            all stocks
        number_held_stocks (int): number of stock in the portfolio
        number_stocks_to_cut (int): number of underperforming stocks to be 
            removed from portfolio monthly

    Returns:
        monthly_ret_df (pandas dataframe): average return of our portfolio
            each month
    """

    df = return_df.copy() # hard copy data frame
    portfolio = [] # initially holding no stocks
    monthly_ret = [0] # initial return is 0 

    # loop over all time
    for i in range(1,len(df)):

        # if we have stocks in portfolio
        if len(portfolio) > 0:

            # calculate month return of stocks
            monthly_ret.append(df[portfolio].iloc[i,:].mean())

            # cut the worst stocks
            bad_stocks = df[portfolio].iloc[i,:].sort_values(ascending=True)[:number_stocks_to_cut].index.values.tolist()

            # update portfolio
            portfolio = [t for t in portfolio if t not in bad_stocks]

        # fill potfolio with stocks with largest monthly returns
        fill = number_held_stocks - len(portfolio)
        new_picks = df.iloc[i,:].sort_values(ascending=False)[:fill].index.values.tolist()
        portfolio = portfolio + new_picks

    # create series of monthly returns of our portfolio
    monthly_ret_df = pd.DataFrame(np.array(monthly_ret),columns=["mon_ret"])

    return monthly_ret_df


def max_dd(return_df, number_held_stocks, number_stocks_to_cut):
    """Calculate max drawdown

    Args:
        return_df (pandas dataframe): dataframe with monthly return info for
            all stocks
        number_held_stocks (int): number of stocks held in the portfolio
        number_stocks_to_cut (int): number of underperforming stocks to be 
            removed monthly

    Returns:
        _type_: _description_
    """
    
    # get monthly return of portfolio
    monthly_ret_df = pflio(return_df,number_held_stocks, number_stocks_to_cut)

    df = monthly_ret_df.copy()
    
    df["cum_return"] = (1 + df["mon_ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd


#calculating overall strategy's KPIs
print("CAGR",CAGR(pflio(return_df,6,3)))
print("sharpe",sharpe(pflio(return_df,6,3),0.025))
print("max_dd",max_dd(pflio(return_df,6,3)))

#calculating KPIs for Index buy and hold strategy over the same period
DJI = yf.download("^DJI",dt.date.today()-dt.timedelta(1900),dt.date.today(),interval='1mo')
DJI["mon_ret"] = DJI["Adj Close"].pct_change()
print("CAGR DJI",CAGR(DJI))
print("sharpe DJI",sharpe(DJI,0.025))
print("max dd DJI",max_dd(DJI))